<a href="https://colab.research.google.com/github/componavt/differential_equations/blob/main/src/hill_equation/97_filtering_t_x_y_Euclidean_values.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🧬📊 Solution Archive for Gene Regulatory ODE System (Compressed) 🚀

**Overview**  
This notebook computes numerical solutions of a gene-regulatory ODE system for many parameter combinations, compresses trajectories using a delta-based filter, and stores all results in a single compact file. Designed for fast and interactive visualization (e.g., in Streamlit) ⚡️.

---

**Model Equations**

\[
\begin{cases}
\frac{dx}{dt} = \frac{K\,x^{1/\alpha}}{b^{1/\alpha} + x^{1/\alpha}} \;-\; \gamma_1\,x,\\[6pt]
\frac{dy}{dt} = \frac{K\,y^{1/\alpha}}{b^{1/\alpha} + y^{1/\alpha}} \;-\; \gamma_2\,y.
\end{cases}
\]

**Fixed parameters**  
- b = 1.0, K = 1.0 ⚙️  
- Time: t ∈ [0, 1.0], N = 500 steps ⏳  
- Initial conditions: x₀ = y₀ = b × (1 + shift), where shift ∈ {−1e−3, −1e−6, 0, 1e−6, 1e−3} 🎯

**Parameter sweeps**  
- α ∈ {1e−9, 1e−10, ..., 1e−14} 🔄  
- γ₁, γ₂ ∈ {0.0, 0.1, ..., 1.0} 🔄  
- Solvers: DOP853, BDF, Radau, RK23 🧮

---

**Compression algorithm** 🗜️  
After solving the ODE:  
- Start from the first point 🚩  
- Keep the next point only if the Euclidean distance in (x, y) exceeds delta = 1e−6 ➡️  
- Always keep the first and last points 🎯  
- This drastically reduces data size while preserving visual fidelity 👁️‍🗨️

---

**Each row in the final DataFrame contains**  
- Parameters: x₀, y₀, γ₁, γ₂, α 📊  
- t: filtered time array (shared across solvers) ⏲️  
- solutions: a dictionary `{method_name: (x_array, y_array)}` 🗂️

All results are saved to:  
📁 `ode_solutions/solutions.pkl` 💾

---

**How to use this in Streamlit**  
- Load the DataFrame with `pd.read_pickle(...)` 📥  
- Let users select α, γ₁, γ₂, and view solver comparisons 🎛️  
- Plot filtered trajectories for better performance ⚡️

---

*This notebook provides a compact and scalable pipeline for collecting and visualizing large numbers of ODE simulations 🚀*

In [9]:
# Modified code for saving ODE solutions with trajectory compression using delta filtering
# Cell 1: Parameters and library imports
import numpy as np
import pandas as pd
from scipy.integrate import solve_ivp
import os

# Model parameters
b = 1.0
t_end = 1.0
K = 1.0
alpha_list = [1e-9 * (10 ** -i) for i in range(6)]

# Time discretization
N = 500
initial_shifts = [-1e-3, -1e-6, 0.0, 1e-6, 1e-3]
t_eval = np.linspace(0, t_end, N)

# Solver methods and gamma parameters
all_methods = ["DOP853", "BDF", "Radau", "RK23"]
gamma_1_list = np.arange(0, 1.01, 0.1)
gamma_2_list = np.arange(0, 1.01, 0.1)
initial_conditions = [(b * (1 + s), b * (1 + s)) for s in initial_shifts]

# Output folder
output_folder = "ode_solutions"
os.makedirs(output_folder, exist_ok=True)

# Delta threshold for filtering # DataFrame with 3630 rows
               # Total filtered points
# delta = 1e-6 # kept: 7188295, skipped: 71705,   percent removed: 0.99%
# delta = 1e-3 # kept: 5878948, skipped: 1381052, percent removed: 19.02%
# delta = 5e-3 # kept: 1658108, skipped: 5601892, percent removed: 77.16%
# delta = 1e-2 # kept: 900358,  skipped: 6359642, percent removed: 87.60%
delta = 5e-2 #   kept: 207087, skipped: 7052913, percent removed: 97.15%

def compress_trajectory(t, x, y, delta):
    indices = [0]
    i = 0
    while i < len(t) - 1:
        j = i + 1
        while j < len(t):
            dist = np.sqrt((x[j] - x[i])**2 + (y[j] - y[i])**2)
            if dist > delta:
                break
            j += 1
        if j >= len(t):
            break
        indices.append(j)
        i = j
    if indices[-1] != len(t) - 1:
        indices.append(len(t) - 1)
    return indices

In [10]:
# Cell 2: Define RHS function
def get_rhs(alpha, gamma1, gamma2):
    def rhs(t, xy):
        x = np.clip(xy[0], 1e-20, 1e20)
        y = np.clip(xy[1], 1e-20, 1e20)
        inv_alpha = 1.0 / alpha
        x_alpha = np.exp(np.clip(np.log(x) * inv_alpha, -700, 700))
        y_alpha = np.exp(np.clip(np.log(y) * inv_alpha, -700, 700))
        b_alpha = np.exp(np.clip(np.log(b) * inv_alpha, -700, 700))
        dxdt = K * x_alpha / (b_alpha + x_alpha) - gamma1 * x
        dydt = K * y_alpha / (b_alpha + y_alpha) - gamma2 * y
        return [dxdt, dydt]
    return rhs

In [11]:
# Cell 3: Solve and compress
records = []
solution_count = 0
skipped_total = 0
kept_total = 0

for x0, y0 in initial_conditions:
    for gamma1 in gamma_1_list:
        for gamma2 in gamma_2_list:
            for alpha in alpha_list:
                for method in all_methods:
                    rhs = get_rhs(alpha, gamma1, gamma2)
                    try:
                        sol = solve_ivp(rhs, [0, t_end], [x0, y0], t_eval=t_eval, method=method)
                        if sol.success:
                            x = sol.y[0]
                            y = sol.y[1]
                            t = sol.t
                            indices = compress_trajectory(t, x, y, delta)
                            x_filt = x[indices]
                            y_filt = y[indices]
                            t_filt = t[indices]

                            kept_total += len(indices)
                            skipped_total += len(t) - len(indices)

                            key = (x0, y0, gamma1, gamma2, alpha)
                            for rec in records:
                                if (rec['x0'], rec['y0'], rec['gamma1'], rec['gamma2'], rec['alpha']) == key:
                                    rec['solutions'][method] = (x_filt, y_filt)
                                    break
                            else:
                                records.append({
                                    'x0': x0,
                                    'y0': y0,
                                    'gamma1': gamma1,
                                    'gamma2': gamma2,
                                    'alpha': alpha,
                                    't': t_filt,
                                    'solutions': {method: (x_filt, y_filt)}
                                })

                            solution_count += 1
                            if solution_count % 1049 == 0:
                                compact = ','.join(map(str, indices[:5])) + ('...' if len(indices) > 5 else '')
                                print(f"#{solution_count}: x0={x0:.1e}, alpha={alpha:.1e}, γ1={gamma1:.2f}, γ2={gamma2:.2f}, {method}, points={len(indices)}, kept={compact}")
                    except Exception as e:
                        print(f"Error: method={method}, alpha={alpha}, gamma1={gamma1}, gamma2={gamma2}, x0={x0}: {e}")

#1049: x0=1.0e+00, alpha=1.0e-13, γ1=0.30, γ2=1.00, DOP853, points=15, kept=0,25,51,78,107...
#2098: x0=1.0e+00, alpha=1.0e-11, γ1=0.70, γ2=1.00, BDF, points=17, kept=0,22,44,67,91...
#3147: x0=1.0e+00, alpha=1.0e-09, γ1=0.00, γ2=1.00, Radau, points=14, kept=0,26,54,83,114...
#4196: x0=1.0e+00, alpha=1.0e-13, γ1=0.40, γ2=0.90, RK23, points=15, kept=0,26,54,83,113...
#5245: x0=1.0e+00, alpha=1.0e-12, γ1=0.80, γ2=0.90, DOP853, points=17, kept=0,22,44,67,91...
#6294: x0=1.0e+00, alpha=1.0e-10, γ1=0.10, γ2=0.90, BDF, points=22, kept=0,21,42,63,85...
#7343: x0=1.0e+00, alpha=1.0e-14, γ1=0.50, γ2=0.80, Radau, points=4, kept=0,157,337,499
#8392: x0=1.0e+00, alpha=1.0e-12, γ1=0.90, γ2=0.80, RK23, points=17, kept=0,22,44,67,91...
#9441: x0=1.0e+00, alpha=1.0e-11, γ1=0.20, γ2=0.80, DOP853, points=16, kept=0,31,62,94,126...
#10490: x0=1.0e+00, alpha=1.0e-09, γ1=0.60, γ2=0.80, BDF, points=8, kept=0,59,122,191,266...
#11539: x0=1.0e+00, alpha=1.0e-13, γ1=1.00, γ2=0.70, Radau, points=6, kept=0,89,19

In [12]:
# Cell 4: Save
df = pd.DataFrame(records)
df.to_pickle(os.path.join(output_folder, "solutions.pkl"))
print(f"Saved DataFrame with {len(df)} rows to 'solutions.pkl'")
print(f"Total filtered points kept: {kept_total}, skipped: {skipped_total}, percent removed: {100.0 * skipped_total / (kept_total + skipped_total):.2f}%")

Saved DataFrame with 3630 rows to 'solutions.pkl'
Total filtered points kept: 207087, skipped: 7052913, percent removed: 97.15%
